<a href="https://colab.research.google.com/github/higegiraffe/wave/blob/master/Wave1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install swan

In [0]:
import wave
import struct
from scipy import fromstring, int16
from scipy import signal
from swan import pycwt
import numpy as np
from pylab import *
%matplotlib inline

In [0]:
# ローカルファイルを直接アップロードする
from google.colab import files
uploaded = files.upload()

In [0]:
ls

In [0]:
# 読み込みするWaveファイル指定

wavfile0 = '07_off.wav'
wavfile1 = '07_af.wav'

In [0]:
#Waveファイルの読み込み

wr0 = wave.open(wavfile0, 'rb')
ch0 = wr0.getnchannels()
width0 = wr0.getsampwidth()
fr0 = wr0.getframerate()
fn0 = wr0.getnframes()

origin0 = wr0.readframes(-1)
# data = origin[:fn * ch * width]
wr0.close()

wr1 = wave.open(wavfile1, 'rb')
ch1 = wr1.getnchannels()
width1 = wr1.getsampwidth()
fr1 = wr1.getframerate()
fn1 = wr1.getnframes()

origin1 = wr1.readframes(-1)
# data = origin[:fn * ch * width]
wr1.close()

In [0]:
#Waveファイル情報表示

print(wavfile0)
print('チャンネル', ch0)
print('サンプルサイズ',width0)
print('フレームレート',fr0)
print('フレーム数', fn0)
print('サンプル時間', fn0 / fr0, '秒')
print('   ')
print(wavfile1)
print('チャンネル', ch1)
print('サンプルサイズ',width1)
print('フレームレート',fr1)
print('フレーム数', fn1)
print('サンプル時間', fn1 / fr1, '秒')

In [0]:
# LRチャンネル分けて、データ読み込み

if width0 == 2:
    X0 = np.frombuffer(origin0, dtype='int16')
    X0 = X0 / 32768.0
elif width0 == 4:
    X0 = np.frombuffer(origin0, dtype='int32')
    X0 = X0 / 2147483648.0
left0 = X0[::2]
right0 = X0[1::2]


if width1 == 2:
    X1 = np.frombuffer(origin1, dtype='int16')
    X1 = X1 / 32768.0
elif width1 == 4:
    X1 = np.frombuffer(origin1, dtype='int32')
    X1 = X1 / 2147483648.0
left1 = X1[::2]
right1 = X1[1::2]

In [0]:
#Wavr波形の表示

length0 = fn0 / fr0                        # 音声の長さ(sec)
time0 = np.arange(0, length0, 1 / fr0)       # 各サンプルに対応する時刻の列

plt.figure(figsize=(16, 10))

plt.subplot(3,1,1)
#plt.figure(figsize=(4, 4))
plt.plot(time0, left0)                    # 音声波形の表示
plt.title('waveform of silent')           # タイトル設定
plt.xlabel('time[s]')                     # 横軸ラベル設定
plt.ylabel('amplitude')                   # 縦軸ラベル設定
#plt.ylim([-0.015, 0.015])                         # 縦軸表示範囲設定
# plt.xlim([0, 0.01])                      # 横軸表示範囲設定

#plt.subplot(2,1,2)
#plt.plot(time, right)
#plt.title('waveform of right')            # タイトル設定
#plt.xlabel('time[s]')                     # 横軸ラベル設定
#plt.ylabel('amplitude')                   # 縦軸ラベル設定
# plt.ylim([-1, 1])                         # 縦軸表示範囲設定
# plt.xlim([1, 1.01])　　　　　　　　　　　# 横軸表示範囲設定

length1 = fn1 / fr1                        # 音声の長さ(sec)
time1 = np.arange(0, length1, 1 / fr1)       # 各サンプルに対応する時刻の列

plt.subplot(3,1,2)
plt.plot(time1, left1)                    # 音声波形の表示
plt.title('waveform of AF')           # タイトル設定
plt.xlabel('time[s]')                     # 横軸ラベル設定
plt.ylabel('amplitude')                   # 縦軸ラベル設定
#plt.ylim([-0.015, 0.015])                         # 縦軸表示範囲設定
# plt.xlim([0, 0.01])                      # 横軸表示範囲設定

plt.subplot(3,1,3)
plt.plot(time0, left0)                    # 音声波形の表示
plt.plot(time1, left1,linewidth = 1,alpha = 0.6)       # 音声波形の表示
plt.title('waveform of silent and AF')           # タイトル設定
plt.xlabel('time[s]')                     # 横軸ラベル設定
plt.ylabel('amplitude')                   # 縦軸ラベル設定
#plt.ylim([-0.015, 0.015])                         # 縦軸表示範囲設定
# plt.xlim([0, 0.01])                      # 横軸表示範囲設定


plt.tight_layout()  # タイトルの被りを防ぐ

#plt.show()
# save as png
plt.savefig('Wave.png')

In [0]:
from google.colab import files
files.download('Wave.png')

In [0]:
#包絡線での波形表示

left0_env = abs(signal.hilbert(left0))
left1_env = abs(signal.hilbert(left1))

plt.figure(figsize=(16, 10))

plt.subplot(3,1,1)
plt.plot(time0,left0)
plt.plot(time0,left0_env,linewidth = 1,alpha = 0.6)
plt.title('envelope of silent')           # タイトル設定
plt.xlabel('time[s]')                     # 横軸ラベル設定
plt.ylabel('amplitude')                   # 縦軸ラベル設定
#plt.ylim([-0.015, 0.015])                         # 縦軸表示範囲設定
#plt.xlim([0, 0.2])                         # 横軸表示範囲設定

plt.subplot(3,1,2)
plt.plot(time1,left1)
plt.plot(time1,left1_env,linewidth = 1,alpha = 0.6)
plt.title('envelope of AF')           # タイトル設定
plt.xlabel('time[s]')                     # 横軸ラベル設定
plt.ylabel('amplitude')                   # 縦軸ラベル設定
#plt.ylim([-0.015, 0.015])                         # 縦軸表示範囲設定
#plt.xlim([0, 0.2])                         # 横軸表示範囲設定

plt.subplot(3,1,3)
plt.plot(time0,left0_env)
plt.plot(time1,left1_env,linewidth = 1,alpha = 0.6)
plt.title('envelope of silent and AF')           # タイトル設定
plt.xlabel('time[s]')                     # 横軸ラベル設定
plt.ylabel('amplitude')                   # 縦軸ラベル設定
#plt.ylim([0, 0.015])                         # 縦軸表示範囲設定
#plt.xlim([0, 0.2])                         # 横軸表示範囲設定

plt.tight_layout()  # タイトルの被りを防ぐ

#plt.show()
# save as png
plt.savefig('Envelope.png')

In [0]:
from google.colab import files
files.download('Envelope.png')

In [0]:
#包絡線での、ピークレベル表示　(dBFS)

#ピーク値を検索
l0_peak_env = max(left0_env)

l1_peak_env = max(left1_env)
  
#ピーク値をdBFS換算
l0_peak_env_dbfs = 20*log(l0_peak_env)/log(10)

l1_peak_env_dbfs = 20*log(l1_peak_env)/log(10)

print('Computing signal envelope')
print('peak of silent dBFS =',l0_peak_env_dbfs)
print('peak of AF dBFS =',l1_peak_env_dbfs)
print('peak of AF-silent dB =',l1_peak_env_dbfs-l0_peak_env_dbfs)

In [0]:
#ピークレベル表示　(dBFS)

#ピーク値を検索
l0_max = max(left0)
l0_min = min(left0)

if abs(l0_max) > abs(l0_min):
  l0_peak = abs(l0_max)
else:
  l0_peak = abs(l0_min)
 
#r_max = max(right)
#r_min = min(right)
#if abs(r_max) > abs(r_min):
#  r_peak = abs(r_max)
#else:
#  r_peak = abs(r_min)

l1_max = max(left1)
l1_min = min(left1)

if abs(l1_max) > abs(l1_min):
  l1_peak = abs(l1_max)
else:
  l1_peak = abs(l1_min)

  
#ピーク値をdBFS換算
l0_peak_dbfs = 20*log(l0_peak)/log(10)
#r_peak_dbfs = 20*log(r_peak)/log(10)

l1_peak_dbfs = 20*log(l1_peak)/log(10)

print('peak of silent dBFS =',l0_peak_dbfs)
print('peak of AF dBFS =',l1_peak_dbfs)
print('peak of AF-silent dB =',l1_peak_dbfs-l0_peak_dbfs)

In [0]:
#波形のフーリエ変換

print('silent 総サンプル数の２のｘ乗 =',(log(len(time0))/log(2)))    #総サンプル数の２のｘ乗
n_00 = 0                                        # サンプリング開始位置(s)
N0 = 2**(int(log(len(time0))/log(2))+1)             # サンプル数

N0_d = int((N0 - (len(time0)))/2)

left0_sq = np.zeros(N0)
left0_sq[N0_d:(N0_d + len(time0))] = left0

G0 = np.fft.fft(left0_sq[n_00:n_00+N0])                  # 高速フーリエ変換
G0_abs = np.abs(G0)                            # 振幅スペクトル
G0_abs_amp = G0_abs / N0 * 2                    # スペクトルを元の振幅にそろえる
flist0 = np.fft.fftfreq(N0, d=1.0/fr0)             # 周波数リスト
print(N0 / fr0)

print('AF 総サンプル数の２のｘ乗 =',(log(len(time1))/log(2)))    #総サンプル数の２のｘ乗
n_01 = 0                                        # サンプリング開始位置(s)
N1 = 2**int(log(len(time1))/log(2)+1)             # サンプル数

N1_d = int((N1 - (len(time1)))/2)

left1_sq = np.zeros(N1)
left1_sq[N1_d:(N1_d + len(time1))] = left1

G1 = np.fft.fft(left1_sq[n_01:n_01+N1])                  # 高速フーリエ変換
G1_abs = np.abs(G1)                            # 振幅スペクトル
G1_abs_amp = G1_abs / N1 * 2                    # スペクトルを元の振幅にそろえる
flist1 = np.fft.fftfreq(N1, d=1.0/fr1)             # 周波数リスト


In [0]:
G0_abs_amp_db = 20*log(G0_abs_amp)/log(10)     # dBFS換算
G1_abs_amp_db = 20*log(G1_abs_amp)/log(10)     # dBFS換算

In [0]:
# 振幅スペクトルを描画

#スペクトル、リニア表示
#plt.plot(flist0[:int(N0/2)], G0_abs_amp[:int(N0/2)], linestyle='-')

#スペクトル、dBFS表示
plt.figure(figsize=(16, 10))

plt.subplot(3,1,1)
plt.plot(flist0[:int(N0/2)], G0_abs_amp_db[:int(N0/2)], linestyle='-')
plt.title('FFT of silent')           # タイトル設定
plt.xlim([1, fr0/2]) 
plt.xlabel("Frequency [Hz]")
plt.ylabel("Amplitude spectrum")
ax = plt.gca()
ax.set_xscale('log')
plt.grid(which="both")
plt.ylim([-180, -40]) 

plt.subplot(3,1,2)
plt.plot(flist1[:int(N1/2)], G1_abs_amp_db[:int(N1/2)], linestyle='-')
plt.title('FFT of AF')           # タイトル設定
plt.xlim([1, fr1/2]) 
plt.xlabel("Frequency [Hz]")
plt.ylabel("Amplitude spectrum")
ax = plt.gca()
ax.set_xscale('log')
plt.grid(which="both")
plt.ylim([-180, -40]) 


plt.subplot(3,1,3)
plt.title('FFT of silent and AF')           # タイトル設定
plt.plot(flist0[:int(N0/2)], G0_abs_amp_db[:int(N0/2)])
plt.xlim([1, fr0/2]) 

plt.plot(flist1[:int(N1/2)], G1_abs_amp_db[:int(N1/2)],linewidth = 1,alpha = 0.6)
#plt.xlim([1, fr1/2]) 

plt.xlabel("Frequency [Hz]")
plt.ylabel("Amplitude spectrum")
ax = plt.gca()
ax.set_xscale('log')
plt.grid(which="both")
plt.ylim([-180, -40]) 

plt.tight_layout()  # タイトルの被りを防ぐ


#plt.show()
# save as png
plt.savefig('FFT.png')

In [0]:
from google.colab import files
files.download('FFT.png')

In [0]:
plt.figure(figsize=(16, 10))

plt.subplot(3,1,1)
plt.plot(flist0[:int(N1/2)], G0_abs_amp_db[:int(N0/2)]-G1_abs_amp_db[:int(N1/2)], linestyle='-')
plt.title('FFT of AF-silent')           # タイトル設定
plt.xlim([1, fr0/2]) 
plt.xlabel("Frequency [Hz]")
plt.ylabel("Amplitude spectrum")
ax = plt.gca()
ax.set_xscale('log')
plt.grid(which="minor")
#plt.ylim([-180, -40]) 

#plt.show()
# save as png
plt.savefig('FFT_delta.png')

In [0]:
from google.colab import files
files.download('FFT_delta.png')

In [0]:
Fs1 = fr1
omega1 = 0.4
# (1)　Freqを指定してcwt
freqs1=np.arange(1,1000,1)
r=pycwt.cwt_f(left1,freqs1,Fs1,pycwt.Morlet(omega1))
rr=np.abs(r)

plt.rcParams['figure.figsize'] = (20, 6)
fig = plt.figure()
ax1 = fig.add_axes([0.1, 0.75, 0.7, 0.2])
ax2 = fig.add_axes([0.1, 0.1, 0.7, 0.60], sharex=ax1)
ax3 = fig.add_axes([0.83, 0.1, 0.03, 0.6])

ax1.plot(time1, left1, 'k')

img = ax2.imshow(np.flipud(rr), extent=[0, time1[-1], freqs1[0], freqs1[-1]],
                 aspect='auto', interpolation='nearest')

twin_ax = ax2
twin_ax.set_yscale('log')
twin_ax.set_xlim(0, time1[-1])
twin_ax.set_ylim(freqs1[0], freqs1[-1])
ax2.tick_params(which='both', labelleft=False, left=False)
twin_ax.tick_params(which='both', labelleft=True, left=True, labelright=False)

fig.colorbar(img, cax=ax3)

#plt.show()
# save as png
plt.savefig('Wavelet.png')

In [0]:
from google.colab import files
files.download('Wavelet.png')